# Some [`socket`](https://docs.python.org/3/library/socket.html) stuff in Python

## Transmitting live data through the Internet
Basically, when we want to transmit data between networked processes, two options are available:

1. [TCP](https://en.wikipedia.org/wiki/Transmission_Control_Protocol): A "reliable" transport protocol with possiblely high transmission latencies. Reliability means that the operating systems will *try* to solve the transmission errors, if they happen, by using [ARQ](https://en.wikipedia.org/wiki/Automatic_repeat_request).
2. [UDP](https://en.wikipedia.org/wiki/User_Datagram_Protocol): An unreliable transport protocol with a minimal (best-effort) transmission latencies. Unreliability means that the operating system will not try to fix any transmission error, if they happen (including the re-ordering of the packets).

## Using UDP

In [1]:
import socket

LISTENING_PORT = 8001

class UDP_receiver():
    # We use a context manager (https://docs.python.org/3/reference/datamodel.html#context-managers).
    def __enter__(self):
        '''Create an UDP socket and listen to it.'''
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        print("socket created")
        self.sock.bind(('', LISTENING_PORT))
        print(f"listening at {self.sock.getsockname()} ... ")
        return self

    def receive(self):
        '''Receive a datagram.'''
        (message, from_addr) = self.sock.recvfrom(1024) # Blocking operation, 1024 is the maximum expected payload size.
        print(f"received {message} from {from_addr}")
        return message
    
    def __exit__(self,ext_type,exc_value,traceback):
        '''Close the socket.'''
        self.sock.close()
        print("socket closed")

class UDP_sender():
    def __enter__(self):
        '''Create an UDP socket.'''
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        print("socket created")
        return self

    def send(self, message, destination):
        '''Send data.'''
        self.sock.sendto(message, destination)
        print(f"message {message} sent to destination {destination}")

    def __exit__(self,ext_type,exc_value,traceback):
        '''Close the socket.'''
        self.sock.close()
        print("socket closed")

def wait_for_a_message():
    with UDP_receiver() as receiver:
        message = receiver.receive().decode("utf-8")
        print(f"recived message = {message}")

def send_message():
    with UDP_sender() as sender:
        message = b"hello world!"
        destination = ('localhost', LISTENING_PORT)
        sender.send(message, destination)
        
import threading

threading.Thread(target=wait_for_a_message).start()
send_message()

socket created
socket created
listening at ('0.0.0.0', 8001) ... 
message b'hello world!' sent to destination ('localhost', 8001)received b'hello world!' from ('127.0.0.1', 37694)

socket closedrecived message = hello world!
socket closed



Notice that UDP is a datagram (independent packet) oriented protocol. The maximum packet size in UDP is 64 KB.

## Using TCP

In [1]:
# This code creates a TCP receiver daemon

import socket

PORT = 8001

class TCP_Receiver():

    # We use a context manager for implementing this "server"
    # (https://docs.python.org/3/reference/datamodel.html#context-managers).
    # This method is run when the "with" context is created (see below).
    def __enter__(self):
        '''Create a TCP socket.'''
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.sock.bind(('', PORT))
        self.sock.listen(1)
        print('waiting for a connection at {} ...'\
              .format(self.sock.getsockname()))
        return self

    def accept(self):
        (self.peer_sock, peer) = self.sock.accept()
        print('\nconnection accepted from {}'.format(peer))

    def receive(self):
        message_final_length = len('hello world!')
        message = bytearray()   # bytearray is mutable and therefore,
                                # faster appending than inmutable types
        while len(message) < message_final_length:
            chunk = self.peer_sock.recv(message_final_length - len(message))
            if chunk == b'':
                raise RuntimeError("socket connection broken")
            message.extend(chunk)
        #print('receivedddd "{}"'.format(message.decode("utf-8")))
        return message
    
    # This method is run when the "with" context is finised (see below).
    def __exit__(self,ext_type,exc_value,traceback):
        self.sock.close()
        print('socket closed')

def receiver_server():
    with TCP_Receiver() as recv:
        recv.accept()
        print('message = {}'.format(recv.receive().decode("utf-8")))
        
import threading

threading.Thread(target=receiver_server).start()

waiting for a connection at ('0.0.0.0', 8001) ...


In [2]:
# Now, we connect to the server.

# 1. Create a TCP socket 
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect(('localhost', PORT))
print('connection established with {}'.format((sock.getpeername())))

# 2. Send a string to the server.
message = b'hello world!'
total_sent = 0 # bytes
while total_sent < len(message):
    sent = sock.send(message[total_sent:])
    if sent == 0:
        raise RuntimeError("socket connection broken")
    total_sent += sent
sock.close()

connection established with ('127.0.0.1', 8001)
connection accepted from ('127.0.0.1', 36920)

message = hello world!
socket closed
